In [7]:
# Import libraries
import tensorflow as tf
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import preprocessing, regularizers

import numpy as np
import os
import time

In [8]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


# Parse Text

In [9]:
# Load file data
path_to_file = tf.keras.utils.get_file('recipies.txt', 'https://raw.githubusercontent.com/AlexBerryhill/RNN-Conference/main/data/recipes/recipes.txt')
text1 = open(path_to_file, 'rb').read().decode(encoding='utf-8')
path_to_file = tf.keras.utils.get_file('conference_text.txt', 'https://raw.githubusercontent.com/AlexBerryhill/RNN-Conference/main/data/conference/conference_talks.txt')
text2 = open(path_to_file, 'rb').read().decode(encoding='utf-8')
text = text1 + text2
print('Length of text: {} characters'.format(len(text)))

40541816/40541816 [==============================] - 8s 0us/step
Length of text: 144063939 characters


In [10]:
# Verify the first part of our data
print(text[:200])

title
ingredients
directions

No-Bake Nut Cookies
["1 c. firmly packed brown sugar", "1/2 c. evaporated milk", "1/2 tsp. vanilla", "1/2 c. broken nuts (pecans)", "2 Tbsp. butter or margarine", "3 1/2 


In [11]:
# Now we'll get a list of the unique characters in the file. This will form the
# vocabulary of our network. There may be some characters we want to remove from this
# set as we refine the network.
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))
print(vocab)

104 unique characters
['\t', '\n', '\r', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '~', 'é', '–', '—', '‘', '’', '“', '”', '…']


In [12]:
# Next, we'll encode encode these characters into numbers so we can use them
# with our neural network, then we'll create some mappings between the characters
# and their numeric representations
ids_from_chars = tf.keras.layers.StringLookup(vocabulary=list(vocab))
chars_from_ids = tf.keras.layers.StringLookup(vocabulary=ids_from_chars.get_vocabulary(), invert=True)

# Here's a little helper function that we can use to turn a sequence of ids
# back into a string:
# turn them into a string:
def text_from_ids(ids):
  joinedTensor = tf.strings.reduce_join(chars_from_ids(ids), axis=-1)
  return joinedTensor.numpy().decode("utf-8")

In [13]:
# Now we'll verify that they work, by getting the code for "A", and then looking
# that up in reverse
testids = ids_from_chars(["T", "r", "u", "t", "h"])
testids

<tf.Tensor: shape=(5,), dtype=int64, numpy=array([56, 85, 88, 87, 75], dtype=int64)>

In [14]:
chars_from_ids(testids)

<tf.Tensor: shape=(5,), dtype=string, numpy=array([b'T', b'r', b'u', b't', b'h'], dtype=object)>

In [15]:
testString = text_from_ids( testids )
testString

'Truth'

In [16]:
# First, create a stream of encoded integers from our text
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(144063939,), dtype=int64, numpy=array([87, 76, 87, ...,  2,  3,  2], dtype=int64)>

In [17]:
# Now, convert that into a tensorflow dataset
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [18]:
# Finally, let's batch these sequences up into chunks for our training
seq_length = 100
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

# This function will generate our sequence pairs:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

# Call the function for every sequence in our list to create a new dataset
# of input->target pairs
dataset = sequences.map(split_input_target)

In [19]:
# Verify our sequences
for input_example, target_example in  dataset.take(1):
    print("Input: ", text_from_ids(input_example))
    print("--------")
    print("Target: ", text_from_ids(target_example))

Input:  title
ingredients
directions

No-Bake Nut Cookies
["1 c. firmly packed brown sugar", "1/2 c. evapora
--------
Target:  itle
ingredients
directions

No-Bake Nut Cookies
["1 c. firmly packed brown sugar", "1/2 c. evaporat


In [20]:
# Finally, we'll randomize the sequences so that we don't just memorize the books
# in the order they were written, then build a new streaming dataset from that.
# Using a streaming dataset allows us to pass the data to our network bit by bit,
# rather than keeping it all in memory. We'll set it to figure out how much data
# to prefetch in the background.

BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [21]:
class ConferenceTextModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super(ConferenceTextModel, self).__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.lstm = tf.keras.layers.LSTM(rnn_units,
                                         return_sequences=True,
                                         return_state=True,
                                         recurrent_initializer='glorot_uniform')
        self.dropout = Dropout(0.5)  # Added Dropout layer here
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = self.embedding(inputs)
        if states is None:
            states = self.lstm.get_initial_state(x)
        x, state_h, state_c = self.lstm(x, initial_state=states, training=training)
        x = self.dropout(x, training=training)  # Apply dropout here
        states = [state_h, state_c]
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else: 
            return x

In [22]:
# Create an instance of our model
vocab_size=len(ids_from_chars.get_vocabulary())
embedding_dim = 256
rnn_units = 1024

model = ConferenceTextModel(vocab_size, embedding_dim, rnn_units)

In [23]:
# Verify the output of our model is correct by running one sample through
# This will also compile the model for us. This step will take a bit.
for input_example_batch, target_example_batch in dataset.take(1):
    states = [tf.zeros([64, model.lstm.units]), tf.zeros([64, model.lstm.units])]
    example_batch_predictions = model(input_example_batch, states=states)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 105) # (batch_size, sequence_length, vocab_size)


In [24]:
# Define the loss function
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

# Compile the model
model.compile(optimizer='adam', loss=loss)

# Define early stopping callback
early_stopping_callback = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(dataset, epochs=1000, callbacks=[early_stopping_callback])

Epoch 1/1000
22287/22287 [==============================] - 241s 11ms/step - loss: 0.7987
Epoch 2/1000
22287/22287 [==============================] - 241s 11ms/step - loss: 0.7211
Epoch 3/1000
22287/22287 [==============================] - 242s 11ms/step - loss: 0.7064
Epoch 4/1000
22287/22287 [==============================] - 240s 11ms/step - loss: 0.7002
Epoch 5/1000
22287/22287 [==============================] - 237s 11ms/step - loss: 0.6974
Epoch 6/1000
22287/22287 [==============================] - 239s 11ms/step - loss: 0.6961
Epoch 7/1000
22287/22287 [==============================] - 243s 11ms/step - loss: 0.6961
Epoch 8/1000
22287/22287 [==============================] - 242s 11ms/step - loss: 0.6970
Epoch 9/1000
22287/22287 [==============================] - 244s 11ms/step - loss: 0.7001
Epoch 10/1000
 1256/22287 [>.............................] - ETA: 3:49 - loss: 0.6324

In [ ]:
# Here's the code we'll use to sample for us. It has some extra steps to apply
# the temperature to the distribution, and to make sure we don't get empty
# characters in our text. Most importantly, it will keep track of our model
# state for us.

class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature=temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "" or "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['','[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices = skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask,validate_indices=False)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states =  self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature

    # Apply the prediction mask: prevent "" or "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Return the characters and model state.
    return chars_from_ids(predicted_ids), states


In [ ]:
# Create an instance of the character generator
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

# Now, let's generate a 1000 character chapter by giving our model "Chapter 1"
# as its starting text
states = None
next_char = tf.constant(['Party Cake'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)

# Print the results formatted.
print(result[0].numpy().decode('utf-8'))


NameError: name 'OneStep' is not defined

In [ ]:
model.save('conference_model2', save_format='tf')

INFO:tensorflow:Assets written to: recipe_model\assets


INFO:tensorflow:Assets written to: recipe_model\assets
